# Data importation

In [75]:
import random
import numpy as np
from sklearn import svm
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
import pandas as pd
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import networkx as nx
import re

%matplotlib inline

In [76]:
info = pd.read_csv(
    "node_information.csv", 
    header= None, 
    names=["Id", "year", "title", "authors", "journal", "abstract"],
    sep=",",
    index_col = 0
)

In [77]:
X_train = pd.read_csv("training_set.txt", sep=" ", header=None)
X_test = pd.read_csv("testing_set.txt", sep=" ", header=None)
y_train = X_train[2]
X_train.drop([2], axis = 1, inplace = True)

print info.shape, X_train.shape, X_test.shape

(27770, 5) (615512, 2) (32648, 2)


In [78]:
######################
### FOR VALIDATION ###
######################


#####################
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2)
#####################

#####################
small_portion_to_train = 50000
small_portion_to_test  = 5000
X_train = X_train[:small_portion_to_train]
y_train = y_train[:small_portion_to_train]

X_test  = X_test[:small_portion_to_test]
y_train = y_test[:small_portion_to_test]
#####################

'small_portion_to_train = 50000\nsmall_portion_to_test  = 5000\nX_train = X_train[:small_portion_to_train]\ny_train = y_train[:small_portion_to_train]\n\nX_test  = X_test[:small_portion_to_test]\ny_train = y_test[:small_portion_to_test]'

# Feature Preprocess

- list_authors is the list of authors in the papers
- list_universities is the list where the authors are from

In [79]:
def universities_to_keep(authors, universities):
    while('(' in authors and ')' in authors):
        universities.append( authors[authors.find('(')+1 : authors.find(')')] )
        authors = authors[: authors.find('(')] + authors[ authors.find(')')+1 : ]
            
    if '(' in authors:
        universities.append( authors[authors.find('(')+1 : ])
        authors = authors[: authors.find('(')]
    
    return authors, universities


def name_to_keep(author):
    if len(author.split(' ')) <= 1:
        return author
    
    while( author[0] == ' ' and len(author) > 0):
        author = author[1:]
    while( author[-1] == ' ' and len(author) > 0):
        author = author[:-1]
    
    author = author.replace('.', '. ')
    author = author.replace('.  ', '. ')
    name_to_keep = author.split(' ')[0][0] + '. ' + author.split(' ')[-1]

    return name_to_keep

# Transform concatenated names of authors to a list of authors 
list_authors = []
list_universities = []

info['authors'] = info['authors'].replace(np.nan, 'missing')
for authors in info['authors']:
    if authors != 'missing':
        ### split the different authors
        authors = authors.lower()
        
        ### Find the universities included in the name
        universities = []
        authors, universities = universities_to_keep(authors, universities)
        
        ### Split the authors
        authors = re.split(',|&', authors)
        
        ### For each author, check if university, and store it. Also, keep just the names (To be improved)
        authors_in_article = []      
        for author in authors:
            if author != ' ':
                authors_in_article.append(name_to_keep(author))
            
        list_universities.append(universities)
        list_authors.append(authors_in_article)
    else:
        list_universities.append(['missing'])
        list_authors.append(['missing'])   
        
info['authors'] = list_authors
info['universities'] = list_universities

# Topologic features

In [80]:
def make_graph(X_train, y_train, X_test):
    X_train = pd.concat([X_train, y_train], axis = 1)
    X_train = X_train.values
    G = nx.DiGraph()
    for i in range(X_train.shape[0]):
        source = X_train[i,0]
        target = X_train[i,1]
        G.add_node(source)
        G.add_node(target)
        if X_train[i,2] == 1:
            G.add_edge(source,target)
            
    X_test = X_test.values
    for i in range(X_test.shape[0]):
        source = X_test[i,0]
        target = X_test[i,1]
        G.add_node(source)
        G.add_node(target)
        
    return G  

In [81]:
G = make_graph(X_train, y_train, X_test)  

In [82]:
def compute_betweeness_array(X, graph):    
    centrality = nx.degree_centrality(graph)  
    centr = [centrality[x[0]] - centrality[x[1]] for x in X[:]]
    return np.array(centr)

In [83]:
def make_common_neighbors(X, G): 
    G2 = G.to_undirected()
    common_neighbors = [len(set(G2.neighbors(x[0])).intersection(G2.neighbors(x[1]))) for x in X[:]]
    return np.array(common_neighbors)

In [84]:
def make_jaccard(X, G):   
    total_jaccard = nx.jaccard_coefficient(G.to_undirected(), [(x[0],x[1]) for x in X[:]])
    jaccard = [jac for u, v, jac in total_jaccard]
    return np.array(jaccard)

In [85]:
#Difference in inlinks between papers
def compute_diff_inlinks(X, graph):
    in_degrees=graph.in_degree()
    diff_deg = [in_degrees[x[1]] - in_degrees[x[0]] for x in X[:]]
    to_deg = [in_degrees[x[1]] for x in X[:]]
    return diff_deg, to_deg

In [86]:
def create_topologic_features(X, G):
    X_ = X.copy()
    X = X.values
    
    X_['Betweeness centrality'] = compute_betweeness_array(X, G)
    X_['Number common neighbours'] = make_common_neighbors(X, G)
    X_['Jaccard coefficienf'] = make_jaccard(X, G)
    diff_deg, to_deg = compute_diff_inlinks(X, G)
    X_['Difference in inlinks coefficient'] = diff_deg
    X_["Number of times to cited"] = to_deg
    
    return X_

In [87]:
%%time
X_train = create_topologic_features(X_train, G)
X_test = create_topologic_features(X_test, G)

Wall time: 2min 12s


# Semantic features
- Cosine similarity within the titles as tf-idf
- Cosine similarity within the abstracts as tf-idf
- Cosine similarity within the titles as word2vec
- Cosine similarity within the abstracts as word2vec

### To try
- Difference cosine similarities?
- Keep the stopwords or not?
- Stemmise the words of not?

# Attribute features

- Difference in publication year
- Number of common authors
- Self-citation
- Same journal
- Number of times "to" cited (Attraction of the "to" paper)

### To try
- Number of times each author of "to" cited [Sum of these number of times] ?
- Number of times each journal cited?
- Number of same university??

In [88]:
def numb_same_authors(authors1, authors2):
    total = 0
    for author in authors1:
        if author in authors2:
            total += 1
    return total

In [89]:
def create_attribute_features(X):
    length = X.shape[0]
    difference_publication_year = np.zeros(length)
    number_same_authors = np.zeros(length)
    self_citation = [False for x in range(length)]
    same_journal = [False for x in range(length)]
    
    ### NOT TO OVERFIT WHEN COUNTING : THE INFORMATION ARE AVAILABLE ONLY FOR X_TRAIN ###
    number_times = X_train.groupby(1).count()
    
    i=-1
    for idx, row in X.iterrows():
        i += 1
        ID1 = int(row[0])
        ID2 = int(row[1])
        
        source = info.loc[ID1]
        target = info.loc[ID2]
        
        ### Difference in publication year
        difference_publication_year[i] = source['year'] - target['year']
        
        ### Number of same authors
        common_authors = numb_same_authors(source['authors'], target['authors'])
        number_same_authors[i] = common_authors
        
        ### Self citation ###
        if common_authors >= 1:
            self_citation[i] = True
            
        ### Same journal ###
        if source['journal'] == target['journal']:
            same_journal[i] = True
            
        
    X_ = X.copy()
    X_['Diff publication'] = difference_publication_year
    X_['Number same authors'] = number_same_authors
    X_['Self citation'] = self_citation
    X_['Same journal'] = same_journal
    return X_

In [90]:
%%time
X_train = create_attribute_features(X_train)
X_test = create_attribute_features(X_test)

Wall time: 12min 3s


In [91]:
X_train.head()

,0,1,Betweeness centrality,Number common neighbours,Jaccard coefficienf,Difference in inlinks coefficient,Number of times to cited,Diff publication,Number same authors,Self citation,Same journal
450485,3299,3256,-0.000648,2,0.040000,9,12,0,0,False,False
408211,105023,9601070,0.001837,0,0.000000,-29,5,5,0,False,False
218450,105302,9511204,0.000216,0,0.000000,15,16,6,0,False,False
20350,202022,9309075,-0.000972,11,0.134146,55,57,9,0,False,False
481477,9807077,9510025,0.000000,0,0.000000,0,0,3,0,False,False


In [92]:
X_test.head()

,0,1,Betweeness centrality,Number common neighbours,Jaccard coefficienf,Difference in inlinks coefficient,Number of times to cited,Diff publication,Number same authors,Self citation,Same journal
416513,101064,5163,0.000072,4,0.111111,13,13,1,1,True,False
517498,12135,9802109,-0.044258,19,0.014961,1245,1249,3,0,False,False
315011,9903196,9806074,-0.002377,37,0.213873,79,125,1,0,False,False
614966,9607008,9308136,-0.000036,1,0.100000,5,6,3,0,False,False
185036,2028,9805105,-0.000612,6,0.070588,32,46,2,1,True,True


# Classifier

In [93]:
def score(pred, real):
    tot = 0
    for i, val in enumerate(real):
        if pred[i] == val:
            tot += 1
    return float(tot)/len(real)

In [97]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = rfc.predict(X_test.drop([0,1], axis = 1))

print score(pred, y_test)

0.939603421525
